<a href="https://colab.research.google.com/github/arloporter/SMNA-Assignment-2/blob/main/graph_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from IPython.display import HTML
from subprocess import getoutput
gpu_type = getoutput('nvidia-smi')
if 'K80' in gpu_type:
  gpu = 'K80'
elif 'T4' in gpu_type:
  gpu = 'T4'
elif 'P100' in gpu_type:
  gpu = 'P100'
display(HTML(f"<h2>{gpu}</h2>"))
print(gpu_type)

Fri Sep 30 05:17:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    35W / 250W |   6339MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install pynvml
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/env-check.py
!pip install --upgrade networkx
!sudo apt-get install graphviz libgraphviz-dev pkg-config
!sudo pip install pygraphviz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'rapidsai-csp-utils' already exists and is not an empty directory.
***********************************************************************
Woo! Your instance has the right kind of GPU, a Tesla P100-PCIE-16GB!
***********************************************************************

Traceback (most recent call last):
  File "/usr/local/bin/pip", line 10, in <module>
  File "/usr/local/lib/python3.7/site-packages/pip/_internal/cli/main.py", line 68, in main
    command = create_command(cmd_name, isolated=("--isolated" in cmd_args))
  File "/usr/local/lib/python3.7/site-packages/pip/_internal/commands/__init__.py", line 114, in create_command
    module = importlib.import_module(module_path)
  File "/usr/local/lib/python3.7/importlib/__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._boots

In [ ]:
!bash rapidsai-csp-utils/colab/update_gcc.sh
import os
os._exit(00)

In [ ]:
import condacolab
condacolab.install()

In [ ]:
import condacolab
condacolab.check()

In [ ]:
!python rapidsai-csp-utils/colab/install_rapids.py stable
import os
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
os.environ['CONDA_PREFIX'] = '/usr/local'

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [2]:
import cudf
import cugraph
import pandas as pd
import networkx as nx

FIXED = True
EXPORTED = True

if not FIXED:   
  stream_df = pd.read_csv('/content/drive/MyDrive/stream_big_edgelist.csv', names=['Source', 'Target', 'Weight', 'Retweet', 'Reply', 'Quote', 'Mention'])
  stream_df['Weight'] = stream_df['Weight'].map(lambda y: [int(x) for x in y.split() if x.isdigit()][0])
  stream_df['Retweet'] = stream_df['Retweet'].map(lambda y: [int(x) for x in y.split() if x.isdigit()][0])
  stream_df['Reply'] = stream_df['Reply'].map(lambda y: [int(x) for x in y.split() if x.isdigit()][0])
  stream_df['Quote'] = stream_df['Quote'].map(lambda y: [int(x) for x in y.split() if x.isdigit()][0])
  stream_df.to_csv('/content/drive/MyDrive/stream_big_edgelist.csv')
if not EXPORTED:
  stream_df = pd.read_csv('/content/drive/MyDrive/stream_big_edgelist.csv', names=['Source', 'Target', 'Weight', 'Retweet', 'Reply', 'Quote', 'Mention'], dtype='string')
  stream_df = stream_df.drop(['Mention'], axis=1)
  net = nx.from_pandas_edgelist(stream_df, source="Source", target="Target", edge_attr='Weight')
  nx.write_gexf(net, "test.gexf")
if FIXED and EXPORTED:  
  graph = nx.read_gexf('/content/drive/MyDrive/test.gexf', )

# Centrality

In [3]:
stream_df = pd.read_csv('/content/drive/MyDrive/stream_big_edgelist.csv', names=['Source', 'Target', 'Weight', 'Retweet', 'Reply', 'Quote', 'Mention'], dtype='string')
stream_df = stream_df.drop(['Mention'], axis=1)
stream_df = stream_df.iloc[1: , :]
print(stream_df.head())
net = cugraph.from_pandas_edgelist(stream_df, source="Source", destination="Target", edge_attr='Weight')

          Source           Target Weight Retweet Reply Quote
0  antti_ellonen       EndGameWW3      1       1     0     0
1     EndGameWW3     CarlaBabbVOA      1       1     0     0
2     EndGameWW3     laraseligman      1       1     0     0
3     EndGameWW3  PentagonPresSec      1       0     0     0
4     EndGameWW3        Liveuamap      1       1     0     0


In [4]:
CALCULATED = True
if not CALCULATED:
  bc = cugraph.betweenness_centrality(net, 100000)
  print(bc)
  bc.to_csv('/content/drive/MyDrive/betweenness_centrality.csv')
elif CALCULATED:
  bc = pd.read_csv('/content/drive/MyDrive/betweenness_centrality.csv')

# Community detection

In [5]:
if not CALCULATED:
  spmmc = cugraph.spectralModularityMaximizationClustering(net, 5)

  spmmc.to_csv('/content/drive/MyDrive/SPMMC.csv')
elif CALCULATED:
  spmmc = cudf.read_csv('/content/drive/MyDrive/SPMMC.csv')
#spmmc = spmmc.loc[:, ~spmmc.columns.str.contains('^Unnamed')]
print(spmmc.head())

   Unnamed: 0  cluster           vertex
0           0        0      SuzietheLip
1           1        0        4USIDream
2           2        0     maggiemae362
3           3        0       AussieYazz
4           4        0  beukes_clifford


In [6]:
if not CALCULATED:
  parts = cugraph.ecg(net)
  parts.to_csv('/content/drive/MyDrive/ECG.csv')
elif CALCULATED:
  parts = cudf.read_csv('/content/drive/MyDrive/ECG.csv')
#parts = parts.loc[:, ~parts.columns.str.contains('^Unnamed')]
print(parts.head())

   Unnamed: 0  partition           vertex
0           0        463      SuzietheLip
1           1        189        4USIDream
2           2         89     maggiemae362
3           3         55       AussieYazz
4           4        317  beukes_clifford


In [7]:
if not CALCULATED:
  sbcc = cugraph.spectralBalancedCutClustering(net, 10)
  sbcc.to_csv('/content/drive/MyDrive/SBCC.csv')
elif CALCULATED:
  sbcc = cudf.read_csv('/content/drive/MyDrive/SBCC.csv')

#sbcc = sbcc.loc[:, ~sbcc.columns.str.contains('^Unnamed')]
print(sbcc.head())

   Unnamed: 0  cluster           vertex
0           0        6      SuzietheLip
1           1        6        4USIDream
2           2        6     maggiemae362
3           3        6       AussieYazz
4           4        6  beukes_clifford


# Seperate tweets from different communities into their own dataframes

In [8]:
tweets_df = cudf.read_csv('/content/drive/MyDrive/ukraine_stream_big.csv')
#!sudo apt install python3-cffi
#!pip install --upgrade cffi==1.15.1

In [ ]:
import time
from IPython.display import clear_output

for community in [0, 1, 2, 3, 4]:
  community_list = []
  community_df = spmmc[spmmc['cluster'] == community]
  for index in range(community_df.shape[0]):
    row = community_df.iloc[index]
    cluster, username = row['cluster'].iloc[0], row['vertex'].iloc[0]
    username_df = tweets_df[tweets_df['author.username'] == username]
    community_list.append(username_df)

  community_total = cudf.concat(community_list)
  print(community_total.head())
  community_total.to_csv(f'/content/drive/MyDrive/communities_SPMMC/{community}/community.csv')

In [ ]:
for community in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]:
  community_list = []
  community_df = sbcc[sbcc['cluster'] == community]
  for index in range(community_df.shape[0]):
    row = community_df.iloc[index]
    cluster, username = row['cluster'].iloc[0], row['vertex'].iloc[0]
    username_df = tweets_df[tweets_df['author.username'] == username]
    community_list.append(username_df)

  community_total = cudf.concat(community_list)
  print(community_total.head())
  community_total.to_csv(f'/content/drive/MyDrive/communities_SBCC/{community}/community.csv')